<a href="https://colab.research.google.com/github/jideexploit3211/Davidbots/blob/ManyLinks/ManyLinks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Step 1: Install Pybullet and imageio-ffmpeg
!pip install pybullet imageio-ffmpeg

# Step 2: Python script for the simulation
import pybullet as p
import pybullet_data
import numpy as np
import imageio_ffmpeg
from base64 import b64encode
from IPython.display import HTML

# Connect to Pybullet in DIRECT mode (no GUI)
physicsClient = p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -10)
planeId = p.loadURDF("plane.urdf")

# Define the box size and position with increased dimensions
boxStartPos = [0, 0, 1]
boxStartOrientation = p.getQuaternionFromEuler([0, 0, 0])
visualShapeId = p.createVisualShape(shapeType=p.GEOM_BOX, halfExtents=[0.5, 0.5, 1.5])
collisionShapeId = p.createCollisionShape(shapeType=p.GEOM_BOX, halfExtents=[0.5, 0.5, 1.5])
boxId = p.createMultiBody(baseMass=1, baseCollisionShapeIndex=collisionShapeId,
                          baseVisualShapeIndex=visualShapeId, basePosition=boxStartPos,
                          baseOrientation=boxStartOrientation)

# Camera parameters
cam_target_pos = [0, 0, 0]
cam_distance = 4
cam_yaw, cam_pitch, cam_roll = 50, -30, 0
cam_width, cam_height = 480, 368
cam_up, cam_up_axis_idx, cam_near_plane, cam_far_plane, cam_fov = [0, 0, 1], 2, 0.01, 100, 60

# Initialize video
vid = imageio_ffmpeg.write_frames('vid.mp4', (cam_width, cam_height), fps=30)
vid.send(None)  # The first frame of the video must be a null frame.

# Simulate the environment and capture frames
for t in range(240):  # Simulate for 240 steps (8 seconds at 30 fps)
    # Update the box position to move up and down
    boxPos = [0, 0, 1 + 0.5 * np.sin(2 * np.pi * t / 60)]  # 0.5 meter amplitude, 2 seconds period
    p.resetBasePositionAndOrientation(boxId, boxPos, boxStartOrientation)

    p.stepSimulation()

    # Capture camera image
    cam_view_matrix = p.computeViewMatrixFromYawPitchRoll(cam_target_pos, cam_distance, cam_yaw, cam_pitch, cam_roll, cam_up_axis_idx)
    cam_projection_matrix = p.computeProjectionMatrixFOV(cam_fov, cam_width / cam_height, cam_near_plane, cam_far_plane)
    image = p.getCameraImage(cam_width, cam_height, cam_view_matrix, cam_projection_matrix)[2][:, :, :3]

    # Add image to video
    vid.send(np.ascontiguousarray(image))

    # Update yaw for a rotating camera effect
    cam_yaw -= 1

# Close video
vid.close()
p.disconnect()

# Display video in notebook
mp4 = open('vid.mp4', 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=480 controls><source src="%s" type="video/mp4"></video>' % data_url)